# Alternative Test 

## Test

In [ ]:
driver.get("https://www.fairprice.com.sg/recipes/")

In [ ]:
html_list = driver.find_elements_by_tag_name('h4')

In [ ]:
html_list

In [ ]:
from selenium.webdriver.common.by import By

In [ ]:
import numpy as np

In [ ]:
urls_list = []
for i in html_list:
    for j in i.find_elements_by_tag_name('a'):
        print(j.get_attribute('href'))

# Actual 

In [2]:
import os 
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import xlsxwriter
from os.path  import basename
import requests
import re

#os.chdir("/Users/Solomonzhs/Desktop/Fresh/YoRipe")
#df = pd.read_excel("/Users/janiceteo/Documents/YoRipe/SongFish/Recipes_Dec 2019 - SongFish.xlsx", "Sheet1")
driver = webdriver.Chrome(ChromeDriverManager().install())
#driver = webdriver.PhantomJS(executable_path='/Users/janiceteo/Downloads/phantomjs-2.1.1-macosx/bin/phantomjs')
#driver = webdriver.Firefox(executable_path = '/Users/janiceteo/Downloads/geckodriver')

Trying to download new driver from http://chromedriver.storage.googleapis.com/80.0.3987.106/chromedriver_win32.zip
Unpack archive C:\Users\User\.wdm\drivers\chromedriver\80.0.3987.106\win32\chromedriver.zip


In [3]:
driver.get("https://www.fairprice.com.sg/recipes/")

## Get URLs 

In [ ]:
for i in driver.find_elements_by_id('load-more-recipes'):
    try:
        #i.click()
        while i in driver.find_elements_by_id('load-more-recipes'):
            i.click()
    except:
        print('Done')

In [ ]:
html_list = driver.find_elements_by_tag_name('h4')
urls_list = []
for i in html_list:
    for j in i.find_elements_by_tag_name('a'):
        urls_list.append(j.get_attribute('href'))

In [ ]:
urls_list = list(dict.fromkeys(urls_list))
urls_list_data = pd.DataFrame(urls_list, columns=['urls'])

In [ ]:
urls_list_data.to_csv(r'C:\Users\Solomonzhs\Desktop\Fresh\YoRipe\fairprice_urls.csv')

# Scraping Actual Pages

In [ ]:
fairprice_recipe = pd.read_csv(r'C:\Users\Solomonzhs\Desktop\Fresh\YoRipe\fairprice_urls.csv')

In [ ]:
recipename = []
servings = []
preptime = []
level_of_difficulty = []
ingredients = []
instructions = []
image_url = []
video_url = []

In [ ]:
def listToString(s):  
    str1 = "\n " 
    return (str1.join(s)) 

In [ ]:
def num_there(s):
    return any(i.isdigit() for i in s)

## Scraping All Pages 

In [ ]:
def edit_string(s):
    s.replace("â€™", ",")
    return s 

In [ ]:
def first_digit(s):
    for i, c in enumerate(str(s)):
        if c.isdigit():
            return c
            break

In [ ]:
import re
def cooking_split(line):
    lines = re.split(r';|,', line)
    return lines

In [1]:
from collections import defaultdict
import re

def humantime2minutes(s):
    d = {
      'w':      7*24*60,
      'week':   7*24*60,
      'weeks':  7*24*60,
      'd':      24*60,
      'day':    24*60,
      'days':   24*60,
      'h':      60,
      'hr':     60,
      'hour':   60,
      'hours':  60,
    }
    mult_items = defaultdict(lambda: 1).copy()
    mult_items.update(d)

    parts = re.search(r'^(\d+)([^\d]*)', str(s).lower().replace(' ', ''))
    if parts:
        return int(parts.group(1)) * mult_items[parts.group(2)] #+ humantime2minutes(re.sub(r'^(\d+)([^\d]*)', '', str(s).lower()))
    else:
        return 0

In [ ]:
for i in fairprice_recipe.urls:
    try:
        driver.get(i)
        
        if driver.find_element_by_tag_name('h4'):
            recipe = edit_string(driver.find_element_by_tag_name('h4').text)
            recipename.append(recipe)
            
        else:
            recipename.append("NA")

        all_units = driver.find_elements_by_tag_name('Strong')
        if all_units:
            serving = first_digit(all_units[0].text)
            servings.append(serving)
            
            if num_there(all_units[1].text):
                lines = cooking_split(all_units[1].text)
                if len(lines) == 2:
                    prep = humantime2minutes(lines[0])
                    preptime.append(prep)
                    cook = humantime2minutes(lines[1])
                    cookingtime.append(cook)
                else:
                    prep = humantime2minutes(lines[0])
                    preptime.append(prep)
                    cookingtime.append("NA")
                level_of_difficulty.append(all_units[2].text)
            else:
                preptime.append("NA")
                cookingtime.append("NA")
                level_of_difficulty.append(all_units[1].text)         
        else:
            servings.append("NA")
            preptime.append("NA")
            level_of_difficulty.append("NA")

        if driver.find_elements_by_class_name('tp-product-left'):
            for j in driver.find_elements_by_class_name('tp-product-left'):
                ingredients.append(j.find_element_by_tag_name('p').text)
        else: 
            ingredients.append("NA")

        instruction_temp = []
        if driver.find_elements_by_class_name('wpb_wrapper'):
            for k in driver.find_elements_by_class_name('wpb_wrapper'):
                for m in k.find_elements_by_tag_name('li'):
                    instruction_temp.append(m.text)
            instruction_next = instruction_temp[0:int(len(instruction_temp)/2)] 
            instructions.append(listToString(instruction_next))
        elif driver.find_element_by_class_name('tp-product-right'):
            instructions.append(driver.find_element_by_class_name('tp-product-right').text)
        else:
            instructions.append("NA")
            
        try:
            src = driver.find_element_by_xpath("//div[@class='tp-video']/img").get_attribute("src")
            image_url.append(src)
            video_url.append("NA")
#             #filename = src.split('/')[-1]
#             #driver.get(src)
#             #driver.save_screenshot(filename)
        except:
            src = driver.find_element_by_xpath("//div[@class='tp-video']/iframe").get_attribute("src")
            image_url.append("NA")
            video_url.append(src)
            
        print(len(recipename))
    
    except:
        print(i)
        recipename.append("NA")
        servings.append("NA")
        preptime.append("NA")
        level_of_difficulty.append("NA")
        ingredients.append("NA")
        instructions.append("NA")
        image_url.append("NA")
        video_url.append("NA")
        continue
        print('Done')

In [ ]:
outcome = pd.DataFrame(list(zip(recipename, servings, preptime, ingredients, instructions, level_of_difficulty, 
                                image_url, video_url)), 
               columns =['recipename', 'servings', 'preptime', 'ingredients', 'instructions', 'level_of_difficulty', 
                         'image_url', 'video_url']) 

In [ ]:
final_outcome = fairprice_recipe.join(outcome)

In [ ]:
final_outcome

In [ ]:
final_outcome.to_csv(r"C:\Users\Solomonzhs\Desktop\Fresh\YoRipe\fairprice_output.csv")

In [ ]:
driver.close()

## Single Build 

In [ ]:
driver.get("https://www.fairprice.com.sg/recipes/fragrant-bitter-gourd-chicken/")

In [ ]:
try:
    src = driver.find_element_by_xpath("//div[@class='tp-video']/img").get_attribute("src")
    image_url.append(src)
    video_url.append("NA")
except:
    src = driver.find_element_by_xpath("//div[@class='tp-video']/iframe").get_attribute("src")
    image_url.append("NA")
    video_url.append(src)

In [ ]:
video_url

In [ ]:
recipename = driver.find_element_by_tag_name('h4').text

In [ ]:
all_units = driver.find_elements_by_tag_name('Strong')
servings = all_units[0].text
preptime = all_units[1].text
level_of_difficulty = all_units[2].text

In [ ]:
for i in driver.find_elements_by_tag_name('Strong'):
    print(i.text)

In [ ]:
for i in driver.find_elements_by_class_name('tp-product-left'):
    ingredients = i.find_element_by_tag_name('p').text       

In [ ]:
instruction_temp = []
if driver.find_elements_by_class_name('wpb_wrapper'):
    for i in driver.find_elements_by_class_name('wpb_wrapper'):
        #print(i.text)
        for j in i.find_elements_by_tag_name('li'):
            instruction_temp.append(j.text)
elif driver.find_element_by_class_name('tp-product-right'):
    print(driver.find_element_by_class_name('tp-product-right').text)
else:
    print("Next")

In [ ]:
instruction_temp

In [ ]:
instruction_next = instruction_temp[0:int(len(instruction_temp)/2)] 

In [ ]:
def listToString(s):  
    str1 = "\n " 
    return (str1.join(s)) 
        
instruction = listToString(instruction_next)
game

### Image Test 

In [ ]:
driver.get("https://www.fairprice.com.sg/recipes/chilli-crab/")

In [ ]:
src = driver.find_element_by_xpath("//div[@class='tp-video']/img").get_attribute("src")

In [ ]:
src

In [ ]:
import requests

url = src
filename = url.split('/')[-1]
r = requests.get(url, allow_redirects=True)
open(filename, 'wb').write(r.content)

In [ ]:
import requests

img_data = requests.get(src).content
with open('image_name.png', 'wb') as handler:
    handler.write(img_data)

In [ ]:
with open(basename(src), "wb") as f:
    f.write(requests.get(src).content)

In [ ]:
import urllib
urllib.request.urlretrieve(src, "00000001.jpg")

In [ ]:
driver.get(src)
filename = url.split('/')[-1]
driver.save_screenshot(filename)

### Video Test  

In [ ]:
driver.get("https://www.fairprice.com.sg/recipes/fragrant-bitter-gourd-chicken/")

In [ ]:
for i in driver.find_elements_by_partial_link_text("youtube"):
    print(i)

In [ ]:
driver.find_elements_by_partial_link_text("youtu")

In [ ]:
driver.find_element_by_xpath("//div[@class='tp-video']/iframe").get_attribute("src")